# Assignment 6: Autoregressive Language Modeling

## Submitted By: Aarathi Vijayachandran (244267)

### Assignment question is available here: https://ovgu-ailab.github.io/lgm2024/assignment6.html

In [1]:
import tensorflow as tf
import numpy as np
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, GRU

BAD GPT

In [37]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Masking, Dropout, LSTM, Attention
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Download and read the Shakespeare dataset
shakespeare_url = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
shakespeare_path = tf.keras.utils.get_file('shakespeare.txt', shakespeare_url)
text = open(shakespeare_path, 'rb').read().decode('utf-8')

# Create a vocabulary from the text
vocab = sorted(set(text))
char_to_index = {char: index for index, char in enumerate(vocab)}
index_to_char = np.array(vocab)

# Convert the text to integer indices
text_as_int = np.array([char_to_index[char] for char in text])
sequence_length = 100
examples_per_epoch = len(text) // (sequence_length + 1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(sequence_length + 1, drop_remainder=True)

# Function to split input and target sequences
def split_input_and_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# Create the dataset by mapping the split function
dataset = sequences.map(split_input_and_target)


In [38]:
# Define hyperparameters
embedding_dim = 512
rnn_units = 2048
batch_size = 256

# Define and build the model
def build_model(vocab_size, embedding_dim, rnn_units):
    inputs = Input(shape=(None,))
    embedding_layer = Embedding(vocab_size, embedding_dim)(inputs)
    masking_layer = Masking(mask_value=0.0)(embedding_layer)
    dropout_layer = Dropout(0.2)(masking_layer)
    lstm_layer = LSTM(rnn_units, return_sequences=True)(dropout_layer)
    attention_layer = Attention()([lstm_layer, lstm_layer])
    outputs = Dense(vocab_size)(attention_layer)
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = build_model(len(vocab), embedding_dim, rnn_units)

# Prepare training data
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(batch_size, drop_remainder=True)

# Compile and train the model
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
model.fit(dataset, epochs=10)


Epoch 1/10
43/43 [==============================] - 52s 1s/step - loss: 3.6024
Epoch 2/10
43/43 [==============================] - 49s 1s/step - loss: 3.2785
Epoch 3/10
43/43 [==============================] - 48s 1s/step - loss: 3.0119
Epoch 4/10
43/43 [==============================] - 48s 1s/step - loss: 2.4406
Epoch 5/10
43/43 [==============================] - 49s 1s/step - loss: 1.6089
Epoch 6/10
43/43 [==============================] - 48s 1s/step - loss: 1.1492
Epoch 7/10
43/43 [==============================] - 47s 1s/step - loss: 1.0246
Epoch 8/10
43/43 [==============================] - 47s 1s/step - loss: 0.9697
Epoch 9/10
43/43 [==============================] - 47s 1s/step - loss: 0.9367
Epoch 10/10
43/43 [==============================] - 49s 1s/step - loss: 0.8666


In [39]:
# Function to generate text using the trained model
def generate_text(model, start_string):
    num_generate = 1000
    input_eval = [char_to_index[s] for s in start_string]
    input_eval = pad_sequences([input_eval], maxlen=sequence_length)
    text_generated = []
    temperature = 1.0

    model.reset_states()
    for _ in range(num_generate):
        predictions = model.predict(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = pad_sequences([[predicted_id]], maxlen=sequence_length)
        text_generated.append(index_to_char[predicted_id])

    return start_string + ''.join(text_generated)

# Generate text with a starting prompt
generated_text = generate_text(model, start_string="ROMEO: ")
print(generated_text)


1/1 [==============================] - 0s 62ms/step
ROMEO: I OOOOOOOOOOOOOOOOOIIIIRRRRRRRonnnnAIRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRIEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEERRRRRRRRRREEEEEEEESSHHEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEENLALLLLLLBBRRRRDDDIRRRRRRRRRRoyyy 
BBBEEEKUUUMMMMMROOOOOOOOOOOOORRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRWMMCDDDRRRREEEEEEEEEEEEEEEvIIVymmmmmmmmmmmmmmmmmmMMMRRRRRRRRRRRRRRREEEEEERRRRRRRRLLGGRRRRRRRRRAAALLLLLLLLLLLLLLLSSrrrrryy.UUSSSSSSSSSSSSSSEEVCCRRRRKKKKKKKKSRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRLAAEEEEEEEEEEERRBORRRRREOOOOOOOOO Prrrrrrrrrrrrrrrruuuuuuu PPBD MEEEEEEEEEVEEEEEEEEEEEEiiit HHAAAAAARRRRRRRRRhiiiiiiiiiiaaaE KDDGGdddddddddd,
WFrrrrrrrrrrrrrrrr:AAAARRRRRRRRRRRRRRRRRRLLLLLBEEERRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRVEEEEEEEEEEEEEEEECiiis,
SSuu:
AHRRRRRRRRKRRRRRRRRRRRRRRRRRiiiiiiiiiiiiiisIe PEEEEEEEEEEEEEEEEEEEEEEEERRRRRRRRIlddddddddddFFEEXEEEEEEEEEEEEEEEEEEPBBGSSSSSSwwwwwwwwwwwwwAAAAAAAAAA BBEEE

Custom RNN--> Not so cool results

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 1s 1us/step


In [3]:
# Read the text file and decode it using UTF-8 encoding
text_content = open(path_to_file, 'rb').read().decode('utf-8')

# Create a sorted list of unique characters from the text
unique_chars = sorted(set(text_content))

# Map characters to indices
char_to_index = {char: index for index, char in enumerate(unique_chars)}

# Map indices to characters
index_to_char = np.array(unique_chars)

# Convert text into an array of integer indices
text_as_indices = np.array([char_to_index[char] for char in text_content])

# Define the length of sequences and generate training examples and targets
sequence_length = 100
examples_per_epoch = len(text_content) // (sequence_length + 1)

# Create a TensorFlow dataset from the integer indices
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_indices)
sequences = char_dataset.batch(sequence_length + 1, drop_remainder=True)

# Function to split input and target text
def split_input_target(sequence):
    input_sequence = sequence[:-1]
    target_sequence = sequence[1:]
    return input_sequence, target_sequence

dataset = sequences.map(split_input_target)

# Hyperparameters
BATCH_SIZE = 64
BUFFER_SIZE = 10000
embedding_dim = 256
rnn_units = 1024

# Shuffle the dataset and batch it
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)

# Define the model
def build_model(vocab_size, embedding_dim, rnn_units):
    model = Sequential([
        Embedding(vocab_size, embedding_dim),
        GRU(rnn_units, return_sequences=True, stateful=False, kernel_initializer='glorot_uniform'),
        Dense(vocab_size)
    ])
    return model

# Initialize the model
model = build_model(vocab_size=len(unique_chars), embedding_dim=embedding_dim, rnn_units=rnn_units)

# Loss function
def compute_loss(labels, logits):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))

# Function to compute validation loss
def validation_loss(model, val_dataset):
    total_loss = 0
    for input_batch, target_batch in val_dataset:
        predictions = model(input_batch)
        total_loss += compute_loss(target_batch, predictions)
    return total_loss / len(val_dataset)

# Optimizer
optimizer = tf.optimizers.Adam(clipnorm=1.0)

# Training epochs
num_epochs = 25

# Training loop
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch + 1}')
    epoch_loss_avg = tf.keras.metrics.Mean()
    for input_batch, target_batch in train_dataset:
        if np.isnan(input_batch.numpy()).any() or np.isnan(target_batch.numpy()).any():
            print('Data contains NaN values')
            break
        with tf.GradientTape() as tape:
            predictions = model(input_batch)
            tf.debugging.check_numerics(predictions, 'Predictions contain NaN or Inf')
            loss = compute_loss(target_batch, predictions)
            tf.debugging.check_numerics(loss, 'Loss contains NaN or Inf')
            epoch_loss_avg.update_state(loss)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(f'Epoch {epoch + 1}: Train Loss: {epoch_loss_avg.result()}')
    print(f'Epoch {epoch + 1}: Val Loss: {validation_loss(model, val_dataset)}')


Starting epoch 1


Epoch 1: Train Loss: 2.682790994644165
Epoch 1: Val Loss: 2.199451446533203
Starting epoch 2
Epoch 2: Train Loss: 2.0244252681732178
Epoch 2: Val Loss: 1.8557013273239136
Starting epoch 3
Epoch 3: Train Loss: 1.7512376308441162
Epoch 3: Val Loss: 1.6321240663528442
Starting epoch 4
Epoch 4: Train Loss: 1.5829226970672607
Epoch 4: Val Loss: 1.4991904497146606
Starting epoch 5
Epoch 5: Train Loss: 1.4769330024719238
Epoch 5: Val Loss: 1.416558861732483
Starting epoch 6
Epoch 6: Train Loss: 1.4035687446594238
Epoch 6: Val Loss: 1.3517584800720215
Starting epoch 7
Epoch 7: Train Loss: 1.349221110343933
Epoch 7: Val Loss: 1.2986985445022583
Starting epoch 8
Epoch 8: Train Loss: 1.3057146072387695
Epoch 8: Val Loss: 1.257363200187683
Starting epoch 9
Epoch 9: Train Loss: 1.2633485794067383
Epoch 9: Val Loss: 1.213409423828125
Starting epoch 10
Epoch 10: Train Loss: 1.2285220623016357
Epoch 10: Val Loss: 1.1796408891677856
Starting epoch 11
Epoch 11: Train Loss: 1.1933982372283936
Epoch 11: V

In [4]:
def generate_text(model, start_string=None, num_generate_chars=1000, temperature=1.0):
    # Convert the starting string to a sequence of indices
    if start_string:
        input_eval = [char_to_index[char] for char in start_string]
        input_eval = tf.expand_dims(input_eval, 0)
    else:
        input_eval = tf.expand_dims([], 0)

    generated_text = []

    model.reset_states()

    for _ in range(num_generate_chars):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions /= temperature

        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        generated_text.append(index_to_char[predicted_id])

        input_eval = tf.expand_dims([predicted_id], 0)

    return ''.join(generated_text)

print(generate_text(model, start_string="C"))


ll tret, the.
Ber wnouth m hathe haleroreey thy yorea t, wn in, ad hand mariter' ll inge e hit,

Smerore h w s liju f h, bomande tste I g wamall t, mout han.
Y ghericeeryorend m s, d cid best thounde! h ndr myo dosthamered pr l bund s waray, a bry?

I fo outil omof temitug hethy w. whoned an.
Sce we ievithevinoth merd lsprd thisto or,
D
ARESThind
ARUS:
ZORouniand onthoun ymores ilor whist ain boutes nt resheseathar nt; bus.
TI ffr-bed buntoldo, se; hated wopownet O! tio ator d tr hane wigrth dour t f amethesthend moul by he no u this wanounoutees ho l yangrs, hey ld othich, thtourem p-mera bes w too.

A foughers orstit s t tand mano he acol ler'thexeryo tierersutherea'stis anallursoubyouththoucis gharick grdouthin n blequromoraldowier s s cly chorourir uthelild macop!
BUSerstougre'stingourthe fe ond f we.
VORING d mbershondyso bunore n am hanore ma' in w y boube m CHEdeat was wnd aro thoug ar thoud ape h ce ft AUSe agho theth t, t ghad supris, s, tour to! ime ld horererde the thor be l

In [5]:
## Tut Based
from tensorflow.keras.utils import to_categorical

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [8]:
# Read the file and decode it for compatibility with Python 2
text = open(path_to_file, 'rb').read().decode('utf-8')

# Print the number of characters in the text
print(f'Text length: {len(text_content)} characters')


Text length: 1115394 characters


In [9]:
# Create a sorted list of unique characters from the text
unique_chars = sorted(set(text))

# Create a StringLookup layer to map characters to indices
char_to_id = tf.keras.layers.StringLookup(
    vocabulary=list(unique_chars), mask_token=None)

# Create an inverted StringLookup layer to map indices back to characters
id_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_id.get_vocabulary(), invert=True, mask_token=None)


In [10]:
def text_from_indices(indices):
    return tf.strings.reduce_join(id_to_char(indices), axis=-1)

all_indices = char_to_id(tf.strings.unicode_split(text, 'UTF-8'))
indices_dataset = tf.data.Dataset.from_tensor_slices(all_indices)

BATCH_SIZE = 128
BUFFER_SIZE = 10000
sequence_length = 50


In [11]:
# Batch the dataset into sequences of fixed length
sequences = indices_dataset.batch(sequence_length + 1, drop_remainder=True)

# Function to split each sequence into input and target text
def split_input_and_target(sequence):
    input_sequence = sequence[:-1]
    target_sequence = sequence[1:]
    return input_sequence, target_sequence

# Map the split function to the dataset
dataset = sequences.map(split_input_and_target)

# Display an example of input and target from the dataset
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_indices(input_example).numpy())
    print("Target:", text_from_indices(target_example).numpy())


Input : b'First Citizen:\nBefore we proceed any further, hear'
Target: b'irst Citizen:\nBefore we proceed any further, hear '


In [12]:
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(128, 50), dtype=tf.int64, name=None), TensorSpec(shape=(128, 50), dtype=tf.int64, name=None))>

In [14]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(char_to_id.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [19]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super(MyModel, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru_layer = tf.keras.layers.GRU(rnn_units,
                                             return_sequences=True,
                                             return_state=True,
                                             recurrent_initializer='glorot_uniform')
        self.dense_layer = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding_layer(inputs, training=training)
        if states is None:
            states = self.gru_layer.get_initial_state(x)
        x, states = self.gru_layer(x, initial_state=states, training=training)
        x = self.dense_layer(x, training=training)

        if return_state:
            return x, states
        else:
            return x


In [20]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [21]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [22]:
model.compile(optimizer='adam', loss=loss)

In [23]:
model.fit(dataset, epochs=10)

Epoch 1/10
170/170 [==============================] - 15s 57ms/step - loss: 2.7436
Epoch 2/10
170/170 [==============================] - 10s 56ms/step - loss: 2.0146
Epoch 3/10
170/170 [==============================] - 10s 54ms/step - loss: 1.7401
Epoch 4/10
170/170 [==============================] - 10s 54ms/step - loss: 1.5806
Epoch 5/10
170/170 [==============================] - 11s 53ms/step - loss: 1.4819
Epoch 6/10
170/170 [==============================] - 10s 54ms/step - loss: 1.4140
Epoch 7/10
170/170 [==============================] - 10s 55ms/step - loss: 1.3616
Epoch 8/10
170/170 [==============================] - 11s 56ms/step - loss: 1.3170
Epoch 9/10
170/170 [==============================] - 11s 55ms/step - loss: 1.2753
Epoch 10/10
170/170 [==============================] - 10s 56ms/step - loss: 1.2344


In [33]:
one_step_model = SingleStepGenerator(model, id_to_char,char_to_id)

TypeError: Exception encountered when calling layer 'string_lookup_1' (type StringLookup).

Cannot convert ['[UNK]'] to EagerTensor of dtype int64

Call arguments received by layer 'string_lookup_1' (type StringLookup):
  • inputs=["'[UNK]'"]

In [34]:
class SingleStepGenerator(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [35]:
one_step_model = SingleStepGenerator(model, id_to_char,char_to_id)

In [36]:
# start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
# end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
# print('\nRun time:', end - start)

ROMEO:
It must be forsworn.

PETRUCHIO:
Gaunt, for a art my wit:
I mean, burn, after, I have like the world and pardon too.
Yet that he will enter a man.

NORTHUMBIRANDA:
Not you so it, I beseech you.

DUKE VINCENTIO:
Here was meet my son, I have rended their grace
To see the thigh or to the barish of stray from his I
knew of great many found,--

MENENIUS:
You so? not so, alize.

GREMIO:
Why, or whom's expedions, with will stand famnidious.

KING EDWARD IV:
Guitly, mother, so must your grace is seeking it.

AUTOLYCUS:
I am more to convey a bloody--accused well.

LORD ROSS:
There, sir.
Partener, 'tis conclunder? nay, scarce.

CURTIS:
Breaks of such a little bright on mine ore a man
may bewits out to shame away:
For it is prieson would have Aurteded o'er the blood flow.
Come, Yorks, remortingly and a met's last.

SAMPSON:
Clarence is so may sure upoin his heads: a fetter that a prince,
and to the other what duthing it eyes
Mies with the great disof their fury of't like the moptal fator;
